在处理数据类型验证的时候，一个现代更常用的库是 pydantic 而不是 dataclass。

- pydantic 可以提供更加高级和严格的数据验证工作：数据类型不对时直接报错，而不是像 dataclass 那样只是 IDE 警告
- pydantic 对现代的 JSON 集成更加友好，可以直接将对象转化为 JSON 数据，或者将 JSON 形式数据转化为 pydantic 对象，或者将 JSON 数据转化为正常的 python 字典
- pydantic 可以通过 validator 装饰器自定义对于字段的验证方式
- pydantic 通过 Field 可以对数据进行更加详尽的限制和描述，使得和很多后端框架结合很好


像 pydantic 和 dataclass 这样的数据验证类，基本都重写了 \_\_init\_\_ 方法，虽然看起来定义的是类属性的数据类型，但是实际上实例化对象的时候，就是按照这样的数据类型要求的

由于重写了 \_\_init\_\_ 方法，所以 pydantic 对实例化时要求必须使用“命名参数”，不能用位置参数来传字段值。并且 pydantic 也重写了 \_\_str\_\_ 方法，使得打印出来的数据更有意义


In [20]:
from pydantic import BaseModel


class Person(BaseModel):
    name: str
    age: int
    game_like: list[str]


zzzongzii: Person = Person(
    name="zzzongzii", age=21, game_like=["Elden Ring", "Sekiro", "Dark Soul III"]
)
print(zzzongzii)

# 获得一个属性
print(zzzongzii.game_like)

name='zzzongzii' age=21 game_like=['Elden Ring', 'Sekiro', 'Dark Soul III']
['Elden Ring', 'Sekiro', 'Dark Soul III']


In [10]:
# Field 关键字定义一些范围要求和基本的描述配合后端框架显示
from pydantic import BaseModel, Field


class Person(BaseModel):
    # ellipsis 表明这个字段必填, examples 可以配合前端显示填写案例
    name: str = Field(..., description="This is person's name", examples=["zzzongzii"])
    age: int = Field(default=18, le=50, ge=18, description="This is person's age")


zzzongzii: Person = Person(name="zzzongzii", age=49)
print(zzzongzii)

name='zzzongzii' age=49


In [18]:
# pydantic 内置了一些数据验证类型
from pydantic import EmailStr, BaseModel


class Person(BaseModel):
    name: str
    email: EmailStr


zzzongzii: Person = Person(name="zzzongzii", email="123@123.com")
print(zzzongzii)

name='zzzongzii' email='123@123.com'


In [ ]:
# 如果我们不满意 pydantic 自带的数据验证，我们可以自己通过 validator 写数据验证函数
# 我们只需要正常抛出异常， pydantic 会自动整理成 ValidationError
from pydantic import BaseModel, field_validator


class Person(BaseModel):
    name: str
    id: int

    @field_validator("id")
    # 可以不用显式声明为 classmethod 但是建议写，并且一定要在最底层
    @classmethod
    # 参数可以任意命名，不需要和 id 一致
    def len_constrict(cls, v):
        if len(str(id)) > 3:
            raise ValueError(f"len(id) is greater than 3 ({v})")


zzzongzii: Person = Person(name="zzzongzii", id=1234)
print(zzzongzii)

ValidationError: 1 validation error for Person
id
  Value error, len(id) is greater than 3 (1234) [type=value_error, input_value=1234, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [41]:
# 与 JSON 数据的联动
from pydantic import BaseModel


class Person(BaseModel):
    name: str
    age: int
    game_like: list[str]


zzzongzii: Person = Person(
    name="zzzongzii", age=21, game_like=["Elden Ring", "Sekiro", "Dark Soul III"]
)

# 转化为 json 形式数据
print(zzzongzii.model_dump_json())
print(type(zzzongzii.model_dump_json()))

# 转化为 dict 形式数据
print(zzzongzii.model_dump())
print(type(zzzongzii.model_dump()))

# 将 json 数据转化为 pydantic 对象
json_str = '{"name": "Jack", "age": "12", "game_like": ["Dark Soul"]}'
json_pydantic = Person.model_validate_json(json_str)  # 自动将 12 转化为了 int 类型
print(json_pydantic)
print(isinstance(json_pydantic, BaseModel))  # 已经转化为了 BaseModel 的实例对象

{"name":"zzzongzii","age":21,"game_like":["Elden Ring","Sekiro","Dark Soul III"]}
<class 'str'>
{'name': 'zzzongzii', 'age': 21, 'game_like': ['Elden Ring', 'Sekiro', 'Dark Soul III']}
<class 'dict'>
name='Jack' age=12 game_like=['Dark Soul']
True
